In [1]:
%cd /content/drive/MyDrive/NLP/Transformers

/content/drive/MyDrive/NLP/Transformers


In [2]:
try:
  from transformer import pipeline
except Exception as e:
  print(e)
  print("installing...")
  !pip install datasets transformers[sentencepiece] -q
  from transformers import pipeline

No module named 'transformer'
installing...


In [3]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [4]:
ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSequenceClassification.from_pretrained(ckpt)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [5]:
example = ["I've been exploring Transformers for Natural Language Processing",
           "I've been exploring Transformers"
]

In [6]:
batch = tokenizer(example, padding=True, return_tensors='pt')
batch

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042, 11131, 19081,  2005,  3019,  2653,
          6364,   102],
        [  101,  1045,  1005,  2310,  2042, 11131, 19081,   102,     0,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [7]:
batch

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042, 11131, 19081,  2005,  3019,  2653,
          6364,   102],
        [  101,  1045,  1005,  2310,  2042, 11131, 19081,   102,     0,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [8]:
batch['labels'] = torch.tensor([1,1])

In [9]:
optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
# doenloading the dataset

from datasets import load_dataset

In [11]:
raw_dataset = load_dataset("glue", "mrpc")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [12]:
!ls ~/.cache/huggingface/datasets

downloads
glue
_root_.cache_huggingface_datasets_glue_mrpc_1.0.0_dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad.lock


In [13]:
raw_dataset_train = raw_dataset['train']
raw_dataset_train

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [14]:
raw_dataset_train.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [15]:
from transformers import AutoTokenizer

In [16]:
ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(ckpt)

In [17]:
tokenized_sentense_1 = tokenizer(raw_dataset['train']['sentence1'])
tokenized_sentense_2 = tokenizer(raw_dataset['train']['sentence2'])

In [18]:
tokenizer("Sentence one", "Sentence two")

{'input_ids': [101, 6251, 2028, 102, 6251, 2048, 102], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [19]:
def tokenizer_func(example):
  return tokenizer(
      example['sentence1'],
      example['sentence2'],
      truncation=True)

In [20]:
tokenized_dataset = raw_dataset.map(tokenizer_func, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [21]:
# Dynamic Padding


In [22]:
from transformers import DataCollatorWithPadding

In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
# Training


In [25]:
from transformers import TrainingArguments

try:
  training_args = TrainingArguments("test-trainer")
  training_args
except:
  !pip install accelerate==0.20.1
  training_args = TrainingArguments("test-trainer")
  training_args

In [26]:
ckpt

'bert-base-uncased'

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(ckpt, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [28]:
from transformers import Trainer


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [29]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.599900
1000,0.447800


TrainOutput(global_step=1377, training_loss=0.4651925911830661, metrics={'train_runtime': 9101.6367, 'train_samples_per_second': 1.209, 'train_steps_per_second': 0.151, 'total_flos': 406183858377360.0, 'train_loss': 0.4651925911830661, 'epoch': 3.0})

In [39]:
predictions = trainer.predict(tokenized_dataset["validation"])
predictions

PredictionOutput(predictions=array([[-2.0216947 ,  2.6161642 ],
       [ 1.3770725 , -1.5489072 ],
       [ 0.54202986, -0.3131326 ],
       [-1.8087684 ,  2.297847  ],
       [ 1.4334191 , -1.6186483 ],
       [-1.8798934 ,  2.4924774 ],
       [-1.8771794 ,  2.4536123 ],
       [-1.970054  ,  2.5779216 ],
       [-1.2400187 ,  1.3722745 ],
       [-1.8581765 ,  2.4555893 ],
       [-2.0280185 ,  2.6173038 ],
       [ 1.437205  , -1.6222988 ],
       [ 1.161532  , -1.2929114 ],
       [-1.8549104 ,  2.4241087 ],
       [-2.025151  ,  2.5957048 ],
       [-1.0138761 ,  0.8670487 ],
       [-2.0258167 ,  2.601831  ],
       [ 1.1187184 , -1.2246493 ],
       [-2.0261574 ,  2.6018474 ],
       [ 1.1748844 , -1.3093845 ],
       [ 1.2570518 , -1.4047494 ],
       [-0.19572961, -0.3293346 ],
       [-0.06728968, -0.5119922 ],
       [-2.0162687 ,  2.612826  ],
       [-2.0185225 ,  2.614506  ],
       [-1.1987897 ,  1.3092505 ],
       [-1.8222615 ,  2.4105935 ],
       [-2.024727  ,  2.59

In [40]:
predictions.predictions

array([[-2.0216947 ,  2.6161642 ],
       [ 1.3770725 , -1.5489072 ],
       [ 0.54202986, -0.3131326 ],
       [-1.8087684 ,  2.297847  ],
       [ 1.4334191 , -1.6186483 ],
       [-1.8798934 ,  2.4924774 ],
       [-1.8771794 ,  2.4536123 ],
       [-1.970054  ,  2.5779216 ],
       [-1.2400187 ,  1.3722745 ],
       [-1.8581765 ,  2.4555893 ],
       [-2.0280185 ,  2.6173038 ],
       [ 1.437205  , -1.6222988 ],
       [ 1.161532  , -1.2929114 ],
       [-1.8549104 ,  2.4241087 ],
       [-2.025151  ,  2.5957048 ],
       [-1.0138761 ,  0.8670487 ],
       [-2.0258167 ,  2.601831  ],
       [ 1.1187184 , -1.2246493 ],
       [-2.0261574 ,  2.6018474 ],
       [ 1.1748844 , -1.3093845 ],
       [ 1.2570518 , -1.4047494 ],
       [-0.19572961, -0.3293346 ],
       [-0.06728968, -0.5119922 ],
       [-2.0162687 ,  2.612826  ],
       [-2.0185225 ,  2.614506  ],
       [-1.1987897 ,  1.3092505 ],
       [-1.8222615 ,  2.4105935 ],
       [-2.024727  ,  2.595037  ],
       [-1.865794  ,

In [41]:
import numpy as np

In [42]:
np.argmax(predictions.predictions)

787

In [43]:
preds = np.argmax(predictions.predictions, axis=-1)
preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [44]:
from datasets import load_metric

In [47]:
metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8382352941176471, 'f1': 0.8858131487889273}

In [48]:
%load_ext tensorboard

In [49]:
%tensorboard --log_dir ./test-trainer/runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-10-05 07:35:22.966104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [50]:
trainer.save_model()

In [51]:
trainer.save_model("./example_model")

In [52]:
loaded_model = AutoModelForSequenceClassification.from_pretrained("./example_model")

In [53]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 

In [54]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b286e60de4ff5b085f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [55]:
from huggingface_hub import notebook_login
notebook_login()

In [56]:
loaded_model.push_to_hub("test-model")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saisubhasish/test-model/commit/1f81eed40b8d7d61b979325dc981d7eab4481090', commit_message='Upload BertForSequenceClassification', commit_description='', oid='1f81eed40b8d7d61b979325dc981d7eab4481090', pr_url=None, pr_revision=None, pr_num=None)